In [23]:
import pandas as pd
import os
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import geopandas as gpd
import contextily as ctx
import matplotlib as mpl
from fiona.io import ZipMemoryFile
import io
import matplotlib.pyplot as plt

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
valid_files = [
    os.path.join("../data/shapefiles", file)
    for file in os.listdir("../data/shapefiles")
    if file.split(".")[-1] == "zip"
]

total_counties_df = []

for county_file in tqdm(sorted(valid_files)):
    county = county_file.split("tl_2020_")[1].split("_")[0]
    # Annoying read using https://gis.stackexchange.com/questions/383465/from-uploaded-zipped-shapefile-to-geopandas-dataframe-in-django-application
    # Just to create a BytesIO object for the demo,
    # similar to your request.FILES['file'].file
    zipshp = io.BytesIO(open(county_file, "rb").read())

    with ZipMemoryFile(zipshp) as memfile:
        with memfile.open() as src:
            crs = src.crs
            county_shape_df = gpd.GeoDataFrame.from_features(src, crs=crs)
    county_shape_df.crs = "epsg:4326"
    total_counties_df.append(county_shape_df)

total_counties_df = pd.concat(total_counties_df)
print(total_counties_df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3221/3221 [17:59<00:00,  2.98it/s]


                                              geometry STATEFP20 COUNTYFP20  \
0    POLYGON ((-86.43344 32.46977, -86.43325 32.469...        01        001   
1    POLYGON ((-86.49678 32.43825, -86.49557 32.439...        01        001   
2    POLYGON ((-86.48140 32.45598, -86.48071 32.455...        01        001   
3    POLYGON ((-86.46499 32.45638, -86.46486 32.457...        01        001   
4    POLYGON ((-86.46751 32.45783, -86.46737 32.459...        01        001   
..                                                 ...       ...        ...   
592  POLYGON ((-66.85314 18.03902, -66.85299 18.039...        72        153   
593  POLYGON ((-66.84973 18.03745, -66.84963 18.037...        72        153   
594  POLYGON ((-66.83711 18.03382, -66.83666 18.034...        72        153   
595  POLYGON ((-66.87492 18.03368, -66.87462 18.033...        72        153   
596  POLYGON ((-66.84237 18.03316, -66.84205 18.033...        72        153   

    TRACTCE20 BLOCKCE20          GEOID20      NAME2

In [9]:
total_counties_df

,geometry,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20
0,"POLYGON ((-86.43344 32.46977, -86.43325 32.469...",01,001,020503,3012,010010205033012,Block 3012,G5040,S,152055,0,+32.4695124,-086.4302842
1,"POLYGON ((-86.49678 32.43825, -86.49557 32.439...",01,001,020600,1009,010010206001009,Block 1009,G5040,S,147523,0,+32.4395167,-086.4931667
2,"POLYGON ((-86.48140 32.45598, -86.48071 32.455...",01,001,020600,3005,010010206003005,Block 3005,G5040,S,30801,0,+32.4546522,-086.4808476
3,"POLYGON ((-86.46499 32.45638, -86.46486 32.457...",01,001,020700,2006,010010207002006,Block 2006,G5040,S,107030,0,+32.4572251,-086.4636602
4,"POLYGON ((-86.46751 32.45783, -86.46737 32.459...",01,001,020700,2008,010010207002008,Block 2008,G5040,S,20638,0,+32.4585243,-086.4667653
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,"POLYGON ((-66.85314 18.03902, -66.85299 18.039...",72,153,750400,1005,721537504001005,Block 1005,G5040,S,17704,0,+18.0393134,-066.8517042
593,"POLYGON ((-66.84973 18.03745, -66.84963 18.037...",72,153,750400,4036,721537504004036,Block 4036,G5040,S,1669,0,+18.0372090,-066.8495140
594,"POLYGON ((-66.83711 18.03382, -66.83666 18.034...",72,153,750102,2020,721537501022020,Block 2020,G5040,S,7599,0,+18.0341955,-066.8363111
595,"POLYGON ((-66.87492 18.03368, -66.87462 18.033...",72,153,750501,2011,721537505012011,Block 2011,G5040,S,14169,0,+18.0327329,-066.8744673


In [13]:
coverage_df = pd.read_csv('../docs/coverage_report.csv.xz', dtype={'GEOID20': object})
coverage_df

,GEOID20,address_count
0,010010201001000,8
1,010010201001001,10
2,010010201001002,14
3,010010201001003,0
4,010010201001004,0
...,...,...
8174950,721537506022011,0
8174951,721537506022012,0
8174952,721537506022013,0
8174953,721537506022014,0


In [14]:
coverage_df['at_least_one'] = coverage_df['address_count'].apply(lambda x: x > 1)
coverage_df

,GEOID20,address_count,at_least_one
0,010010201001000,8,True
1,010010201001001,10,True
2,010010201001002,14,True
3,010010201001003,0,False
4,010010201001004,0,False
...,...,...,...
8174950,721537506022011,0,False
8174951,721537506022012,0,False
8174952,721537506022013,0,False
8174953,721537506022014,0,False


In [15]:
cdf = total_counties_df.merge(coverage_df, on=['GEOID20'], how='outer')
cdf

,geometry,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,address_count,at_least_one
0,"POLYGON ((-86.43344 32.46977, -86.43325 32.469...",01,001,020503,3012,010010205033012,Block 3012,G5040,S,152055,0,+32.4695124,-086.4302842,46,True
1,"POLYGON ((-86.49678 32.43825, -86.49557 32.439...",01,001,020600,1009,010010206001009,Block 1009,G5040,S,147523,0,+32.4395167,-086.4931667,6,True
2,"POLYGON ((-86.48140 32.45598, -86.48071 32.455...",01,001,020600,3005,010010206003005,Block 3005,G5040,S,30801,0,+32.4546522,-086.4808476,14,True
3,"POLYGON ((-86.46499 32.45638, -86.46486 32.457...",01,001,020700,2006,010010207002006,Block 2006,G5040,S,107030,0,+32.4572251,-086.4636602,34,True
4,"POLYGON ((-86.46751 32.45783, -86.46737 32.459...",01,001,020700,2008,010010207002008,Block 2008,G5040,S,20638,0,+32.4585243,-086.4667653,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8174950,"POLYGON ((-66.85314 18.03902, -66.85299 18.039...",72,153,750400,1005,721537504001005,Block 1005,G5040,S,17704,0,+18.0393134,-066.8517042,0,False
8174951,"POLYGON ((-66.84973 18.03745, -66.84963 18.037...",72,153,750400,4036,721537504004036,Block 4036,G5040,S,1669,0,+18.0372090,-066.8495140,0,False
8174952,"POLYGON ((-66.83711 18.03382, -66.83666 18.034...",72,153,750102,2020,721537501022020,Block 2020,G5040,S,7599,0,+18.0341955,-066.8363111,0,False
8174953,"POLYGON ((-66.87492 18.03368, -66.87462 18.033...",72,153,750501,2011,721537505012011,Block 2011,G5040,S,14169,0,+18.0327329,-066.8744673,0,False


In [16]:
# cdf.to_file('../data/us_blocks_address.geojson')

In [17]:
# tdf = cdf.sample(int(len(cdf)/100))
# tdf

,geometry,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,address_count,at_least_one
3955051,"POLYGON ((-90.82762 32.37308, -90.82761 32.373...",28,149,950901,2029,281499509012029,Block 2029,G5040,S,48098,0,+32.3727584,-090.8267060,0,False
7583790,"POLYGON ((-82.84076 36.65833, -82.84071 36.658...",51,169,030200,2052,511690302002052,Block 2052,G5040,S,460952,0,+36.6597146,-082.8368425,1,False
5957002,"POLYGON ((-77.13827 39.91950, -77.13720 39.919...",42,001,030200,2132,420010302002132,Block 2132,G5040,S,71101,0,+39.9181164,-077.1362359,0,False
5807495,"POLYGON ((-95.93846 36.36583, -95.93823 36.365...",40,143,005500,1048,401430055001048,Block 1048,G5040,S,48322,0,+36.3644632,-095.9375184,0,False
1845242,"POLYGON ((-83.55395 32.75882, -83.55384 32.758...",13,289,060200,1000,132890602001000,Block 1000,G5040,S,124323,0,+32.7394076,-083.5428698,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702178,"POLYGON ((-84.12355 33.71491, -84.12287 33.715...",13,089,023317,3003,130890233173003,Block 3003,G5040,S,320533,0,+33.7160055,-084.1185033,240,True
1039608,"POLYGON ((-104.85336 39.73291, -104.85300 39.7...",08,005,007400,2000,080050074002000,Block 2000,G5040,S,165586,0,+39.7312910,-104.8496032,22,True
7579301,"POLYGON ((-79.62192 37.87027, -79.62168 37.870...",51,163,930200,3107,511639302003107,Block 3107,G5040,S,1561858,0,+37.8741494,-079.6082815,0,False
6026353,"POLYGON ((-79.97079 40.88551, -79.97077 40.885...",42,019,902900,1028,420199029001028,Block 1028,G5040,S,55731,0,+40.8859467,-079.9677776,0,False


# Merge the geometries, so that each county has at least one value?

In [45]:
cdf

,geometry,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,address_count,at_least_one
0,"POLYGON ((-86.43344 32.46977, -86.43325 32.469...",01,001,020503,3012,010010205033012,Block 3012,G5040,S,152055,0,+32.4695124,-086.4302842,46,True
1,"POLYGON ((-86.49678 32.43825, -86.49557 32.439...",01,001,020600,1009,010010206001009,Block 1009,G5040,S,147523,0,+32.4395167,-086.4931667,6,True
2,"POLYGON ((-86.48140 32.45598, -86.48071 32.455...",01,001,020600,3005,010010206003005,Block 3005,G5040,S,30801,0,+32.4546522,-086.4808476,14,True
3,"POLYGON ((-86.46499 32.45638, -86.46486 32.457...",01,001,020700,2006,010010207002006,Block 2006,G5040,S,107030,0,+32.4572251,-086.4636602,34,True
4,"POLYGON ((-86.46751 32.45783, -86.46737 32.459...",01,001,020700,2008,010010207002008,Block 2008,G5040,S,20638,0,+32.4585243,-086.4667653,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8174950,"POLYGON ((-66.85314 18.03902, -66.85299 18.039...",72,153,750400,1005,721537504001005,Block 1005,G5040,S,17704,0,+18.0393134,-066.8517042,0,False
8174951,"POLYGON ((-66.84973 18.03745, -66.84963 18.037...",72,153,750400,4036,721537504004036,Block 4036,G5040,S,1669,0,+18.0372090,-066.8495140,0,False
8174952,"POLYGON ((-66.83711 18.03382, -66.83666 18.034...",72,153,750102,2020,721537501022020,Block 2020,G5040,S,7599,0,+18.0341955,-066.8363111,0,False
8174953,"POLYGON ((-66.87492 18.03368, -66.87462 18.033...",72,153,750501,2011,721537505012011,Block 2011,G5040,S,14169,0,+18.0327329,-066.8744673,0,False


In [40]:
import matplotlib.colors as clrs
cmap = clrs.ListedColormap(['red', 'green'])

In [42]:
from matplotlib.pyplot import figure
figure(figsize=(10, 10))

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [ ]:
cdf.to_crs("EPSG:4326")
ax = cdf.plot(alpha=1, column='at_least_one', cmap=cmap, legend=True)
ctx.add_basemap(ax, crs=cdf.crs.to_string(), source=ctx.providers.Stamen.Toner)
ax.set_xlim([-130, -60])
ax.set_ylim([20, 55])
plt.savefig('../docs/us.pdf')